In [1]:
import pandas as pd
import numpy as np


In [2]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

In [3]:
!pip install sklearn --upgrade

In [4]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [5]:
data = "../Resources/modelling16.csv"
modelling = pd.read_csv(data)
modelling

,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD
0,B1,Y,23,M,721,72.0,Medium,RTW,2021-04-26,2021-01-20
1,B1,Y,56,M,526,256.0,Government,RTW,NaN,2021-04-14
2,AV,Y,43,M,721,95.0,Medium,RTW,2021-05-25,2021-02-08
3,AV,Y,69,M,406,259.0,Small,RTW,2021-10-25,2021-07-22
4,C2,N,64,M,703,28.0,Small,No RTW,2021-08-28,2021-01-01
...,...,...,...,...,...,...,...,...,...,...
3013,AV,N,50,F,219,10.0,Small,No RTW,NaN,2021-02-26
3014,C1,Y,37,F,702,99.0,Small,RTW,2021-08-23,2021-03-02
3015,C1,N,59,F,703,403.0,Small,No RTW,NaN,2021-07-05
3016,B1,Y,25,M,203,6.0,Small,RTW,2021-09-27,2021-07-13


In [6]:
model_df = modelling.dropna(subset=['Status+3weeks'])

In [7]:
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
model_df.Gender = le1.fit_transform(model_df.Gender)
# model_df.Physical_or_Mental_Injury = le2.fit_transform(model_df.Physical_or_Mental_Injury)
model_df.Employer_Type = le3.fit_transform(model_df.Employer_Type)
model_df

C:\Users\Guest1\anaconda3\lib\site-packages\pandas\core\generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD
0,B1,Y,23,1,721,72.0,3,RTW,2021-04-26,2021-01-20
1,B1,Y,56,1,526,256.0,1,RTW,NaN,2021-04-14
2,AV,Y,43,1,721,95.0,3,RTW,2021-05-25,2021-02-08
3,AV,Y,69,1,406,259.0,4,RTW,2021-10-25,2021-07-22
4,C2,N,64,1,703,28.0,4,No RTW,2021-08-28,2021-01-01
...,...,...,...,...,...,...,...,...,...,...
3013,AV,N,50,0,219,10.0,4,No RTW,NaN,2021-02-26
3014,C1,Y,37,0,702,99.0,4,RTW,2021-08-23,2021-03-02
3015,C1,N,59,0,703,403.0,4,No RTW,NaN,2021-07-05
3016,B1,Y,25,1,203,6.0,4,RTW,2021-09-27,2021-07-13


In [8]:
X = model_df.drop(["Working", "Status+3weeks", "ARD", 'RTW_Date', "Team"], axis=1)
y = model_df["Working"]
#print(X.shape, y.shape)
X.shape

(2567, 5)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_train.shape

(1925, 5)

In [10]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [12]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [13]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 21,002
Trainable params: 21,002
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=150,
    shuffle=True,
    verbose=2
)

Epoch 1/150
61/61 - 0s - loss: 0.6745 - accuracy: 0.6026
Epoch 2/150
61/61 - 0s - loss: 0.6497 - accuracy: 0.6255
Epoch 3/150
61/61 - 0s - loss: 0.6389 - accuracy: 0.6416
Epoch 4/150
61/61 - 0s - loss: 0.6307 - accuracy: 0.6561
Epoch 5/150
61/61 - 0s - loss: 0.6269 - accuracy: 0.6530
Epoch 6/150
61/61 - 0s - loss: 0.6231 - accuracy: 0.6665
Epoch 7/150
61/61 - 0s - loss: 0.6192 - accuracy: 0.6483
Epoch 8/150
61/61 - 0s - loss: 0.6218 - accuracy: 0.6556
Epoch 9/150
61/61 - 0s - loss: 0.6216 - accuracy: 0.6649
Epoch 10/150
61/61 - 0s - loss: 0.6132 - accuracy: 0.6732
Epoch 11/150
61/61 - 0s - loss: 0.6154 - accuracy: 0.6681
Epoch 12/150
61/61 - 0s - loss: 0.6125 - accuracy: 0.6629
Epoch 13/150
61/61 - 0s - loss: 0.6148 - accuracy: 0.6597
Epoch 14/150
61/61 - 0s - loss: 0.6155 - accuracy: 0.6670
Epoch 15/150
61/61 - 0s - loss: 0.6127 - accuracy: 0.6738
Epoch 16/150
61/61 - 0s - loss: 0.6101 - accuracy: 0.6670
Epoch 17/150
61/61 - 0s - loss: 0.6119 - accuracy: 0.6623
Epoch 18/150
61/61 - 0s

Epoch 142/150
61/61 - 0s - loss: 0.5748 - accuracy: 0.6904
Epoch 143/150
61/61 - 0s - loss: 0.5745 - accuracy: 0.6883
Epoch 144/150
61/61 - 0s - loss: 0.5736 - accuracy: 0.6842
Epoch 145/150
61/61 - 0s - loss: 0.5716 - accuracy: 0.6940
Epoch 146/150
61/61 - 0s - loss: 0.5747 - accuracy: 0.6894
Epoch 147/150
61/61 - 0s - loss: 0.5725 - accuracy: 0.6909
Epoch 148/150
61/61 - 0s - loss: 0.5755 - accuracy: 0.6836
Epoch 149/150
61/61 - 0s - loss: 0.5803 - accuracy: 0.6816
Epoch 150/150
61/61 - 0s - loss: 0.5708 - accuracy: 0.6956


In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

21/21 - 0s - loss: 0.6439 - accuracy: 0.6589
Normal Neural Network - Loss: 0.6438528299331665, Accuracy: 0.65887850522995


In [18]:
encoded_predictions = model.predict_classes(X_test_scaled[:30])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [19]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:30])}")

Predicted classes: ['Y' 'Y' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y'
 'N' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'Y']
Actual Labels: ['N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'N', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y']


In [20]:
##Model rtw outcomes
predict_df = modelling[modelling['Status+3weeks'].isna()]
#predict_df = predict_df[predict_df['RTW_Date'].isna()]
predict_df


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD
1818,C2,N,49,M,703,133.0,Small,NaN,NaN,2021-08-24
2126,C2,Y,52,F,229,56.0,Small,NaN,2021-07-06,2021-08-20
2533,C1,N,16,M,149,187.0,Small,NaN,NaN,2021-09-03
2558,A2,N,38,M,459,6.0,Medium,NaN,NaN,2021-08-16
2559,DH,Y,59,F,542,6.0,Large,NaN,2021-10-11,2021-08-16
...,...,...,...,...,...,...,...,...,...,...
3005,DH,N,23,M,459,64.0,Medium,NaN,NaN,2021-09-30
3006,A2,N,31,M,459,17.0,Medium,NaN,NaN,2021-09-30
3007,A1,N,36,F,422,49.0,Small,NaN,NaN,2021-09-30
3008,C1,N,57,M,459,20.0,Small,NaN,NaN,2021-09-30


In [21]:
predict_df.Gender = le1.fit_transform(predict_df.Gender)
# predict_df.Physical_or_Mental_Injury = le2.fit_transform(predict_df.Physical_or_Mental_Injury)
predict_df.Employer_Type = le3.fit_transform(predict_df.Employer_Type)
predict_df.head(10)

C:\Users\Guest1\anaconda3\lib\site-packages\pandas\core\generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD
1818,C2,N,49,1,703,133.0,4,NaN,NaN,2021-08-24
2126,C2,Y,52,0,229,56.0,4,NaN,2021-07-06,2021-08-20
2533,C1,N,16,1,149,187.0,4,NaN,NaN,2021-09-03
2558,A2,N,38,1,459,6.0,3,NaN,NaN,2021-08-16
2559,DH,Y,59,0,542,6.0,2,NaN,2021-10-11,2021-08-16
2560,C3,Y,25,0,219,8.0,2,NaN,2021-11-02,2021-08-16
2561,A1,Y,23,1,119,8.0,2,NaN,2021-12-13,2021-08-16
2562,AV,N,31,1,703,17.0,0,NaN,NaN,2021-08-16
2563,DH,N,50,0,542,56.0,4,NaN,NaN,2021-08-16
2564,AV,Y,26,1,459,17.0,4,NaN,2021-08-18,2021-08-16


In [22]:
predict = predict_df.drop(["Working", "Status+3weeks", "ARD", 'RTW_Date', "Team"], axis=1)

In [23]:
predict.head()

,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type
1818,49,1,703,133.0,4
2126,52,0,229,56.0,4
2533,16,1,149,187.0,4
2558,38,1,459,6.0,3
2559,59,0,542,6.0,2


In [24]:
predict_scaler = MinMaxScaler().fit(predict)
predict_test_scaled = predict_scaler.transform(predict)
predict_test_scaled

array([[0.53225806, 1.        , 0.67808219, 0.09554598, 1.        ],
       [0.58064516, 0.        , 0.1369863 , 0.04022989, 1.        ],
       [0.        , 1.        , 0.0456621 , 0.13433908, 1.        ],
       ...,
       [0.32258065, 0.        , 0.35730594, 0.03520115, 1.        ],
       [0.66129032, 1.        , 0.39954338, 0.01436782, 1.        ],
       [0.58064516, 1.        , 0.4760274 , 0.01508621, 0.25      ]])

In [25]:
encoded_predictions = model.predict_classes(predict_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [26]:
predict_df = predict_df.drop(['Working'], axis=1)
predict_df['Working'] = prediction_labels
predict_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD,Working
1818,C2,49,1,703,133.0,4,NaN,NaN,2021-08-24,N
2126,C2,52,0,229,56.0,4,NaN,2021-07-06,2021-08-20,N
2533,C1,16,1,149,187.0,4,NaN,NaN,2021-09-03,N
2558,A2,38,1,459,6.0,3,NaN,NaN,2021-08-16,Y
2559,DH,59,0,542,6.0,2,NaN,2021-10-11,2021-08-16,Y
...,...,...,...,...,...,...,...,...,...,...
3005,DH,23,1,459,64.0,3,NaN,NaN,2021-09-30,Y
3006,A2,31,1,459,17.0,3,NaN,NaN,2021-09-30,Y
3007,A1,36,0,422,49.0,4,NaN,NaN,2021-09-30,Y
3008,C1,57,1,459,20.0,4,NaN,NaN,2021-09-30,N


In [27]:
predict_df.Gender = le1.inverse_transform(predict_df.Gender)
# predict_df.Physical_or_Mental_Injury = le2.inverse_transform(predict_df.Physical_or_Mental_Injury)
predict_df.Employer_Type = le3.inverse_transform(predict_df.Employer_Type)
predict_df.head()

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD,Working
1818,C2,49,M,703,133.0,Small,NaN,NaN,2021-08-24,N
2126,C2,52,F,229,56.0,Small,NaN,2021-07-06,2021-08-20,N
2533,C1,16,M,149,187.0,Small,NaN,NaN,2021-09-03,N
2558,A2,38,M,459,6.0,Medium,NaN,NaN,2021-08-16,Y
2559,DH,59,F,542,6.0,Large,NaN,2021-10-11,2021-08-16,Y


In [28]:
model_df.Gender = le1.inverse_transform(model_df.Gender)
# model_df.Physical_or_Mental_Injury = le2.inverse_transform(model_df.Physical_or_Mental_Injury)
model_df.Employer_Type = le3.inverse_transform(model_df.Employer_Type)
model_df

C:\Users\Guest1\anaconda3\lib\site-packages\pandas\core\generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD
0,B1,Y,23,M,721,72.0,Medium,RTW,2021-04-26,2021-01-20
1,B1,Y,56,M,526,256.0,Government,RTW,NaN,2021-04-14
2,AV,Y,43,M,721,95.0,Medium,RTW,2021-05-25,2021-02-08
3,AV,Y,69,M,406,259.0,Small,RTW,2021-10-25,2021-07-22
4,C2,N,64,M,703,28.0,Small,No RTW,2021-08-28,2021-01-01
...,...,...,...,...,...,...,...,...,...,...
3013,AV,N,50,F,219,10.0,Small,No RTW,NaN,2021-02-26
3014,C1,Y,37,F,702,99.0,Small,RTW,2021-08-23,2021-03-02
3015,C1,N,59,F,703,403.0,Small,No RTW,NaN,2021-07-05
3016,B1,Y,25,M,203,6.0,Small,RTW,2021-09-27,2021-07-13


In [29]:
predict_df.head()

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD,Working
1818,C2,49,M,703,133.0,Small,NaN,NaN,2021-08-24,N
2126,C2,52,F,229,56.0,Small,NaN,2021-07-06,2021-08-20,N
2533,C1,16,M,149,187.0,Small,NaN,NaN,2021-09-03,N
2558,A2,38,M,459,6.0,Medium,NaN,NaN,2021-08-16,Y
2559,DH,59,F,542,6.0,Large,NaN,2021-10-11,2021-08-16,Y


In [30]:
working_df = modelling[modelling['Status+3weeks'].isna()]
working_df = working_df.dropna(subset=['RTW_Date'])
working_df

,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD
2126,C2,Y,52,F,229,56.0,Small,NaN,2021-07-06,2021-08-20
2559,DH,Y,59,F,542,6.0,Large,NaN,2021-10-11,2021-08-16
2560,C3,Y,25,F,219,8.0,Large,NaN,2021-11-02,2021-08-16
2561,A1,Y,23,M,119,8.0,Large,NaN,2021-12-13,2021-08-16
2564,AV,Y,26,M,459,17.0,Small,NaN,2021-08-18,2021-08-16
...,...,...,...,...,...,...,...,...,...,...
2996,A2,Y,48,M,459,47.0,Medium,NaN,2021-11-17,2021-09-29
2999,AV,Y,33,F,459,12.0,Emergency Services,NaN,2021-10-21,2021-09-30
3000,DH,Y,43,M,422,22.0,Small,NaN,2021-10-18,2021-09-30
3002,A1,Y,45,F,531,11.0,Large,NaN,2021-10-13,2021-09-30


In [31]:
merged_df = pd.concat([predict_df, model_df])
merged_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD,Working
1818,C2,49,M,703,133.0,Small,NaN,NaN,2021-08-24,N
2126,C2,52,F,229,56.0,Small,NaN,2021-07-06,2021-08-20,N
2533,C1,16,M,149,187.0,Small,NaN,NaN,2021-09-03,N
2558,A2,38,M,459,6.0,Medium,NaN,NaN,2021-08-16,Y
2559,DH,59,F,542,6.0,Large,NaN,2021-10-11,2021-08-16,Y
...,...,...,...,...,...,...,...,...,...,...
3013,AV,50,F,219,10.0,Small,No RTW,NaN,2021-02-26,N
3014,C1,37,F,702,99.0,Small,RTW,2021-08-23,2021-03-02,Y
3015,C1,59,F,703,403.0,Small,No RTW,NaN,2021-07-05,N
3016,B1,25,M,203,6.0,Small,RTW,2021-09-27,2021-07-13,Y


In [32]:
merged_df["ARD"] = merged_df["ARD"].apply(pd.to_datetime)
merged_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD,Working
1818,C2,49,M,703,133.0,Small,NaN,NaN,2021-08-24,N
2126,C2,52,F,229,56.0,Small,NaN,2021-07-06,2021-08-20,N
2533,C1,16,M,149,187.0,Small,NaN,NaN,2021-09-03,N
2558,A2,38,M,459,6.0,Medium,NaN,NaN,2021-08-16,Y
2559,DH,59,F,542,6.0,Large,NaN,2021-10-11,2021-08-16,Y
...,...,...,...,...,...,...,...,...,...,...
3013,AV,50,F,219,10.0,Small,No RTW,NaN,2021-02-26,N
3014,C1,37,F,702,99.0,Small,RTW,2021-08-23,2021-03-02,Y
3015,C1,59,F,703,403.0,Small,No RTW,NaN,2021-07-05,N
3016,B1,25,M,203,6.0,Small,RTW,2021-09-27,2021-07-13,Y


In [33]:
merged_df["ARD"] = merged_df["ARD"].dt.strftime("%m/%y")
merged_df.dtypes

Team              object
Age                int64
Gender            object
Injury_Code        int64
Lodgement_Gap    float64
Employer_Type     object
Status+3weeks     object
RTW_Date          object
ARD               object
Working           object
dtype: object

In [34]:
merged_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Employer_Type,Status+3weeks,RTW_Date,ARD,Working
1818,C2,49,M,703,133.0,Small,NaN,NaN,08/21,N
2126,C2,52,F,229,56.0,Small,NaN,2021-07-06,08/21,N
2533,C1,16,M,149,187.0,Small,NaN,NaN,09/21,N
2558,A2,38,M,459,6.0,Medium,NaN,NaN,08/21,Y
2559,DH,59,F,542,6.0,Large,NaN,2021-10-11,08/21,Y
...,...,...,...,...,...,...,...,...,...,...
3013,AV,50,F,219,10.0,Small,No RTW,NaN,02/21,N
3014,C1,37,F,702,99.0,Small,RTW,2021-08-23,03/21,Y
3015,C1,59,F,703,403.0,Small,No RTW,NaN,07/21,N
3016,B1,25,M,203,6.0,Small,RTW,2021-09-27,07/21,Y


In [35]:
merged_df_rtw = merged_df.loc[(merged_df['Working']=="Y")]

monthly_rtw = merged_df_rtw.groupby(["ARD"])
month_rtw_count = monthly_rtw['Working'].count()
monthly_rtw_df = pd.DataFrame({"RTW":month_rtw_count}).reset_index()

merged_df_nortw = merged_df.loc[(merged_df['Working']=="N")]

monthly_nortw = merged_df_nortw.groupby(["ARD"])
month_nortw_count = monthly_nortw['Working'].count()
monthly_nortw_df = pd.DataFrame({"No_RTW":month_nortw_count}).reset_index()


In [36]:
merged_monthly_df = monthly_rtw_df.merge(monthly_nortw_df, how="outer", on="ARD")
merged_monthly_df

,ARD,RTW,No_RTW
0,01/21,149,104
1,02/21,232,133
2,03/21,238,150
3,04/21,180,136
4,05/21,218,135
5,06/21,194,142
6,07/21,241,153
7,08/21,223,125
8,09/21,152,113


In [37]:
merged_monthly_df["rtw_pct"] = merged_monthly_df["RTW"] / (merged_monthly_df["RTW"] + merged_monthly_df["No_RTW"])
merged_monthly_df

,ARD,RTW,No_RTW,rtw_pct
0,01/21,149,104,0.588933
1,02/21,232,133,0.635616
2,03/21,238,150,0.613402
3,04/21,180,136,0.569620
4,05/21,218,135,0.617564
5,06/21,194,142,0.577381
6,07/21,241,153,0.611675
7,08/21,223,125,0.640805
8,09/21,152,113,0.573585


In [38]:
merged_monthly_df.to_csv('../Resources/final16.csv', index=False)